In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.functional as F
from torch.autograd import Variable

In [2]:
# Hyper Parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST Dataset 
train_dataset = dsets.MNIST(root='../../dataset/mnist', 
                            train=True, 
                            transform=transforms.ToTensor(),  
                            download=True)

test_dataset = dsets.MNIST(root='../../dataset/mnist', 
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [5]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [7]:
net = Net(input_size, hidden_size, num_classes)
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [11]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Convert torch tensor to Variable
        images = Variable(images.view(-1, 28*28)) # 100,28,28 -> 100,784
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Step [100/600], Loss: 0.0239
Epoch [1/5], Step [200/600], Loss: 0.1238
Epoch [1/5], Step [300/600], Loss: 0.0435
Epoch [1/5], Step [400/600], Loss: 0.0215
Epoch [1/5], Step [500/600], Loss: 0.0070
Epoch [1/5], Step [600/600], Loss: 0.0429
Epoch [2/5], Step [100/600], Loss: 0.0235
Epoch [2/5], Step [200/600], Loss: 0.0217
Epoch [2/5], Step [300/600], Loss: 0.0051
Epoch [2/5], Step [400/600], Loss: 0.0183
Epoch [2/5], Step [500/600], Loss: 0.0096
Epoch [2/5], Step [600/600], Loss: 0.0060
Epoch [3/5], Step [100/600], Loss: 0.0084
Epoch [3/5], Step [200/600], Loss: 0.0156
Epoch [3/5], Step [300/600], Loss: 0.0241
Epoch [3/5], Step [400/600], Loss: 0.0523
Epoch [3/5], Step [500/600], Loss: 0.0900
Epoch [3/5], Step [600/600], Loss: 0.0156
Epoch [4/5], Step [100/600], Loss: 0.0046
Epoch [4/5], Step [200/600], Loss: 0.0053
Epoch [4/5], Step [300/600], Loss: 0.0369
Epoch [4/5], Step [400/600], Loss: 0.0248
Epoch [4/5], Step [500/600], Loss: 0.0115
Epoch [4/5], Step [600/600], Loss:

In [12]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

# Save the Model
torch.save(net.state_dict(), 'model.pkl')

Accuracy of the network on the 10000 test images: 97 %
